In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
data_path = 'sample_labels.csv'
NIH_Data = pd.read_csv(data_path)

In [3]:
NIH_Data[:1]

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,0.139


In [4]:
NIH_Data["Finding Labels"][0].split('|')[0] == 'Emphysema'

True

In [5]:
train_data = NIH_Data[:2803] #50% of total data
test_data = NIH_Data[2803:5046] #40% of total data
validation_data = NIH_Data[5046:5607] #10% of total data

In [6]:
def getTarget(data):
    target = np.zeros([13])
    split_data = data.split('|')
    for i in range(0,len(split_data)):
        if(split_data[i] == 'Cardiomegaly'):
            target[0]=1
        if(split_data[i] == 'Emphysema'):
            target[1]=1
        if(split_data[i] == 'Atelectasis'):
            target[2]=1
        if(split_data[i] == 'Infiltration'):
            target[3]=1
        if(split_data[i] == 'Effusion'):
            target[4]=1
        if(split_data[i] == 'Pneumothorax'):
            target[5]=1
        if(split_data[i] == 'Pleural_Thickening'):
            target[6]=1
        if(split_data[i] == 'Edema'):
            target[7]=1
        if(split_data[i] == 'Mass'):
            target[8]=1
        if(split_data[i] == 'Nodule'):
            target[9]=1
        if(split_data[i] == 'Consolidation'):
            target[10]=1
        if(split_data[i] == 'Fibrosis'):
            target[11]=1
        if(split_data[i] == 'No Finding'):
            target[12]=1
            
    return target

In [7]:
def getDataTarget(data,start_val):
    data_targets = np.zeros([data.shape[0],13])
    for i in range(0,data.shape[0]):
        data_targets[i,:] = getTarget(data["Finding Labels"][i+start_val])
    return data_targets

In [8]:
def load_data(data,start_val):
    #print(data.shape[0])
    data_files = ["" for x in range(data.shape[0])]
    for i in range(0,data.shape[0]):
        #print(data["Image Index"][i+start_val])
        data_files[i] = ('images/'+data["Image Index"][i+start_val])
    data_files = np.asarray(data_files)
    #print(data_files)
    data_targets = getDataTarget(data,start_val)
    return data_files, data_targets

In [9]:
train_files, train_targets = load_data(train_data,0)
valid_files, valid_targets = load_data(validation_data,5046)
test_files, test_targets = load_data(test_data,2803)

In [10]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [11]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 2243/2243 [00:35<00:00, 62.51it/s]


In [13]:
train_tensors.shape

(2803, 224, 224, 3)

In [12]:
VGG16_model = keras.applications.vgg16.VGG16()
#VGG16_model.summary()

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
for layer in VGG16_model.layers:
    model.add(layer)
#model.summary()

In [14]:
model.layers.pop()
#model.summary()

In [15]:
#for layer in VGG16_model.layers:
 #   layer.trainable = False;

In [17]:
model.add(Dense(13, activation='softmax'))

In [18]:
for layer in model.layers[:10]:
    layer.trainable = False

In [23]:
from keras import optimizers

sgd = optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
model.fit(train_tensors, train_targets,
          batch_size=16,
          epochs=10,
          shuffle=True,
          verbose=1,
          validation_data=(valid_tensors, valid_targets),
          )

Train on 2803 samples, validate on 560 samples
Epoch 1/10
2803/2803 [==============================] - 116s - loss: 2.8246 - acc: 0.5448 - val_loss: 2.8456 - val_acc: 0.4946
Epoch 2/10
2803/2803 [==============================] - 113s - loss: 2.4305 - acc: 0.5637 - val_loss: 2.7142 - val_acc: 0.4946
Epoch 3/10
2803/2803 [==============================] - 114s - loss: 2.3387 - acc: 0.5637 - val_loss: 2.6886 - val_acc: 0.4946
Epoch 4/10
2803/2803 [==============================] - 114s - loss: 2.3152 - acc: 0.5637 - val_loss: 2.6716 - val_acc: 0.4946
Epoch 5/10
2803/2803 [==============================] - 114s - loss: 2.3034 - acc: 0.5637 - val_loss: 2.6587 - val_acc: 0.4946
Epoch 6/10
2803/2803 [==============================] - 114s - loss: 2.2963 - acc: 0.5637 - val_loss: 2.6508 - val_acc: 0.4946
Epoch 7/10
2803/2803 [==============================] - 114s - loss: 2.2918 - acc: 0.5637 - val_loss: 2.6436 - val_acc: 0.4946
Epoch 8/10
2803/2803 [==============================] - 114s - l

In [27]:
predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)
score = log_loss(Y_valid, predictions_valid)

NameError: name 'X_valid' is not defined

In [17]:
#from keras import optimizers
#Adam = optimizers.Adam(lr=0.001)
#model.compile(optimizer=Adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
from keras.callbacks import ModelCheckpoint  
epochs = 50
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

In [20]:
model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),epochs=epochs, 
          batch_size=10, callbacks=[checkpointer], verbose=1)

Train on 2803 samples, validate on 560 samples
Epoch 1/50
2803/2803 [==============================] - 262s - loss: 2.8446 - acc: 0.5623 - val_loss: 2.9128 - val_acc: 0.4946
Epoch 2/50
2803/2803 [==============================] - 257s - loss: 2.5378 - acc: 0.5637 - val_loss: 2.7231 - val_acc: 0.4946
Epoch 3/50
2803/2803 [==============================] - 258s - loss: 2.3819 - acc: 0.5637 - val_loss: 2.6509 - val_acc: 0.4946
Epoch 4/50
2803/2803 [==============================] - 256s - loss: 2.3170 - acc: 0.5637 - val_loss: 2.6313 - val_acc: 0.4946
Epoch 5/50
2803/2803 [==============================] - 256s - loss: 2.2948 - acc: 0.5637 - val_loss: 2.6270 - val_acc: 0.4946
Epoch 6/50
2803/2803 [==============================] - 257s - loss: 2.2871 - acc: 0.5637 - val_loss: 2.6266 - val_acc: 0.4946
Epoch 7/50
2803/2803 [==============================] - 257s - loss: 2.2842 - acc: 0.5637 - val_loss: 2.6258 - val_acc: 0.4946
Epoch 8/50
2803/2803 [==============================] - 238s - l

KeyboardInterrupt: 

In [35]:
# get index of predicted dog breed for each image in test set
image_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(image_predictions)==np.argmax(test_targets, axis=1))/len(image_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 52.9202%
